In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 128
epochs = 300
start_epoch = 0
train_batch = 100
test_batch = 200
lr = 0.04
schedule = [75,150,225]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style2/128/b0/siamese' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

# iterative training
feedback = 0
iter_time = [1000, 2000, 2500]

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.01

# constrastive
thresh = 0.4

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
class SiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None,should_invert=True):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        self.should_invert = should_invert
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            while True:
                #keep looping till a different class image is found
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] !=img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
#         img0 = img0.convert("L")
#         img1 = img1.convert("L")
        
        if self.should_invert:
            img0 = PIL.ImageOps.invert(img0)
            img1 = PIL.ImageOps.invert(img1)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(img1_tuple[1]!=img0_tuple[1])],dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [8]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

# Dataset

In [9]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')

train_aug = transforms.Compose([
#     transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_ = SiameseNetworkDataset(datasets.ImageFolder(train_dir), transform=train_aug, should_invert=False)
train_loader = DataLoader(train_, shuffle=True, num_workers=num_workers, batch_size=train_batch)
val_ = SiameseNetworkDataset(datasets.ImageFolder(val_dir), transform=val_aug, should_invert=False)
val_loader = DataLoader(val_, shuffle=True, num_workers=num_workers, batch_size=test_batch)

# Model

In [10]:
model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.0, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    pt = torch.load(pretrained)['state_dict']
    model.load_state_dict(pt)

In [11]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.17M


# Loss

In [12]:
criterion = ContrastiveLoss(margin=1.0).cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

# Train

In [14]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs0, inputs1, targets) in enumerate(train_loader):
        batch_size = inputs0.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()

        
        outputs0 = model(inputs0)
        outputs1 = model(inputs1)
        
        loss = criterion(outputs0, outputs1, targets)
            
        # compute output
        outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

        # measure accuracy and record loss
        pred = outputs.data
        pred[pred < thresh] = 0.
        pred[pred >= thresh] = 1.
        prec1 = [accuracy_score(targets.data.cpu().numpy(), pred.cpu().numpy())]
        
        losses.update(loss.data.tolist(), inputs0.size(0))
        top1.update(prec1[0], inputs0.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
        print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    with torch.no_grad():
        for batch_idx, (inputs0, inputs1, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs0, inputs1, targets = inputs0.cuda(), inputs1.cuda(), targets.cuda()

            # compute output
            outputs0 = model(inputs0)
            outputs1 = model(inputs1)
            loss = criterion(outputs0, outputs1, targets)
            outputs = F.pairwise_distance(outputs0, outputs1, keepdim=True)

            # measure accuracy and record loss
            pred = outputs.data
            pred[pred < thresh] = 0.
            pred[pred >= thresh] = 1.
            prec1 = [accuracy_score(targets.data.cpu().numpy(), pred.cpu().numpy())]
            losses.update(loss.data.tolist(), inputs0.size(0))
            top1.update(prec1[0], inputs0.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            # plot progress
            bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                        batch=batch_idx + 1,
                        size=len(val_loader),
                        data=data_time.avg,
                        bt=batch_time.avg,
                        total=bar.elapsed_td,
                        eta=bar.eta_td,
                        loss=losses.avg,
                        top1=top1.avg,)
            bar.next()
        print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
             batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
        bar.finish()
    return (losses.avg, top1.avg)

In [16]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)

    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 300] LR: 0.040000
1/689 Data:1.362 | Batch:2.920 | Total:0:00:02 | ETA:0:33:30 | Loss:1.066001534461975 | top1:0.59
2/689 Data:0.011 | Batch:0.292 | Total:0:00:03 | ETA:0:18:24 | Loss:1.1326550245285034 | top1:0.535
3/689 Data:0.007 | Batch:0.268 | Total:0:00:03 | ETA:0:13:16 | Loss:1.1582169930140178 | top1:0.47
4/689 Data:0.016 | Batch:0.285 | Total:0:00:03 | ETA:0:10:45 | Loss:0.9972808808088303 | top1:0.4725
5/689 Data:0.009 | Batch:0.278 | Total:0:00:04 | ETA:0:09:14 | Loss:0.8643649697303772 | top1:0.476
6/689 Data:0.008 | Batch:0.285 | Total:0:00:04 | ETA:0:08:13 | Loss:0.7676416685183843 | top1:0.475
7/689 Data:0.007 | Batch:0.273 | Total:0:00:04 | ETA:0:07:29 | Loss:0.7009154047284808 | top1:0.4742857142857143
8/689 Data:0.008 | Batch:0.285 | Total:0:00:04 | ETA:0:06:57 | Loss:0.6483923494815826 | top1:0.48
9/689 Data:0.018 | Batch:0.277 | Total:0:00:05 | ETA:0:06:31 | Loss:0.6093002491527133 | top1:0.47444444444444445
10/689 Data:0.009 | Batch:0.271 | Total:0:00:

74/689 Data:0.011 | Batch:0.274 | Total:0:00:23 | ETA:0:02:47 | Loss:0.31966468027314626 | top1:0.49635135135135133
75/689 Data:0.020 | Batch:0.268 | Total:0:00:23 | ETA:0:02:47 | Loss:0.319544607202212 | top1:0.49546666666666667
76/689 Data:0.010 | Batch:0.273 | Total:0:00:23 | ETA:0:02:47 | Loss:0.3189016696261732 | top1:0.4955263157894737
77/689 Data:0.011 | Batch:0.273 | Total:0:00:23 | ETA:0:02:46 | Loss:0.31819249225127233 | top1:0.4967532467532468
78/689 Data:0.012 | Batch:0.273 | Total:0:00:24 | ETA:0:02:47 | Loss:0.3178838841043986 | top1:0.49756410256410255
79/689 Data:0.012 | Batch:0.268 | Total:0:00:24 | ETA:0:02:46 | Loss:0.3172181661747679 | top1:0.4978481012658228
80/689 Data:0.018 | Batch:0.273 | Total:0:00:24 | ETA:0:02:46 | Loss:0.31642528865486386 | top1:0.49825
81/689 Data:0.009 | Batch:0.273 | Total:0:00:24 | ETA:0:02:46 | Loss:0.3163850982615977 | top1:0.49790123456790125
82/689 Data:0.013 | Batch:0.268 | Total:0:00:25 | ETA:0:02:45 | Loss:0.31559930432860445 | to

146/689 Data:0.010 | Batch:0.266 | Total:0:00:42 | ETA:0:02:28 | Loss:0.2986433095719716 | top1:0.4999315068493151
147/689 Data:0.010 | Batch:0.278 | Total:0:00:43 | ETA:0:02:28 | Loss:0.2984363414397856 | top1:0.4998639455782313
148/689 Data:0.011 | Batch:0.279 | Total:0:00:43 | ETA:0:02:28 | Loss:0.29825235017248103 | top1:0.49986486486486487
149/689 Data:0.011 | Batch:0.268 | Total:0:00:43 | ETA:0:02:27 | Loss:0.2982041851786159 | top1:0.4999328859060403
150/689 Data:0.015 | Batch:0.278 | Total:0:00:43 | ETA:0:02:28 | Loss:0.2981106303135554 | top1:0.49933333333333335
151/689 Data:0.011 | Batch:0.271 | Total:0:00:44 | ETA:0:02:28 | Loss:0.2978511503990123 | top1:0.49973509933774835
152/689 Data:0.010 | Batch:0.265 | Total:0:00:44 | ETA:0:02:27 | Loss:0.29777291946505247 | top1:0.4993421052631579
153/689 Data:0.018 | Batch:0.266 | Total:0:00:44 | ETA:0:02:27 | Loss:0.29764875613786035 | top1:0.4987581699346405
154/689 Data:0.011 | Batch:0.277 | Total:0:00:44 | ETA:0:02:26 | Loss:0.29

217/689 Data:0.011 | Batch:0.278 | Total:0:01:02 | ETA:0:02:10 | Loss:0.2893897778427546 | top1:0.49737327188940095
218/689 Data:0.011 | Batch:0.246 | Total:0:01:02 | ETA:0:02:08 | Loss:0.2892433261379189 | top1:0.4974770642201835
219/689 Data:0.009 | Batch:0.290 | Total:0:01:02 | ETA:0:02:07 | Loss:0.2891033260517469 | top1:0.49753424657534245
220/689 Data:0.011 | Batch:0.284 | Total:0:01:03 | ETA:0:02:07 | Loss:0.2890578413551504 | top1:0.49736363636363634
221/689 Data:0.010 | Batch:0.278 | Total:0:01:03 | ETA:0:02:07 | Loss:0.2888510283167006 | top1:0.49778280542986425
222/689 Data:0.011 | Batch:0.276 | Total:0:01:03 | ETA:0:02:08 | Loss:0.28878047236719645 | top1:0.4974774774774775
223/689 Data:0.018 | Batch:0.266 | Total:0:01:03 | ETA:0:02:08 | Loss:0.28871003708764575 | top1:0.49730941704035875
224/689 Data:0.011 | Batch:0.282 | Total:0:01:04 | ETA:0:02:08 | Loss:0.28859608919758883 | top1:0.4975
225/689 Data:0.011 | Batch:0.260 | Total:0:01:04 | ETA:0:02:08 | Loss:0.288516888022

289/689 Data:0.011 | Batch:0.273 | Total:0:01:21 | ETA:0:01:49 | Loss:0.28325957946711344 | top1:0.49871972318339103
290/689 Data:0.013 | Batch:0.279 | Total:0:01:22 | ETA:0:01:49 | Loss:0.28318749039337554 | top1:0.4987931034482759
291/689 Data:0.011 | Batch:0.271 | Total:0:01:22 | ETA:0:01:49 | Loss:0.2832184574243539 | top1:0.4986254295532646
292/689 Data:0.010 | Batch:0.277 | Total:0:01:22 | ETA:0:01:49 | Loss:0.2831655393724572 | top1:0.4986301369863014
293/689 Data:0.012 | Batch:0.278 | Total:0:01:23 | ETA:0:01:48 | Loss:0.2831284022371924 | top1:0.49897610921501706
294/689 Data:0.012 | Batch:0.280 | Total:0:01:23 | ETA:0:01:49 | Loss:0.2830664817573262 | top1:0.4992517006802721
295/689 Data:0.011 | Batch:0.268 | Total:0:01:23 | ETA:0:01:48 | Loss:0.28295961123401836 | top1:0.4993559322033898
296/689 Data:0.011 | Batch:0.273 | Total:0:01:23 | ETA:0:01:48 | Loss:0.28287342200810844 | top1:0.49929054054054056
297/689 Data:0.011 | Batch:0.266 | Total:0:01:24 | ETA:0:01:48 | Loss:0.2

...

101/689 Data:0.010 | Batch:0.552 | Total:0:00:53 | ETA:0:05:25 | Loss:0.0065080033368713425 | top1:0.9925742574257426
102/689 Data:0.009 | Batch:0.553 | Total:0:00:53 | ETA:0:05:24 | Loss:0.006547905155243557 | top1:0.9925490196078431
103/689 Data:0.008 | Batch:0.560 | Total:0:00:54 | ETA:0:05:24 | Loss:0.006584356427975383 | top1:0.9924271844660194
104/689 Data:0.009 | Batch:0.551 | Total:0:00:55 | ETA:0:05:24 | Loss:0.006541550204019586 | top1:0.9925
105/689 Data:0.008 | Batch:0.561 | Total:0:00:55 | ETA:0:05:24 | Loss:0.006611754062067207 | top1:0.9924761904761905
106/689 Data:0.010 | Batch:0.552 | Total:0:00:56 | ETA:0:05:24 | Loss:0.006623635384628114 | top1:0.9924528301886792
107/689 Data:0.009 | Batch:0.550 | Total:0:00:56 | ETA:0:05:23 | Loss:0.006619511398274091 | top1:0.9924299065420561
108/689 Data:0.009 | Batch:0.562 | Total:0:00:57 | ETA:0:05:22 | Loss:0.0067547729616274374 | top1:0.9923148148148148
109/689 Data:0.009 | Batch:0.543 | Total:0:00:57 | ETA:0:05:21 | Loss:0.00

172/689 Data:0.008 | Batch:0.553 | Total:0:01:32 | ETA:0:04:47 | Loss:0.006285743775447901 | top1:0.9926744186046511
173/689 Data:0.009 | Batch:0.551 | Total:0:01:33 | ETA:0:04:46 | Loss:0.0063392408768346295 | top1:0.9926011560693642
174/689 Data:0.009 | Batch:0.555 | Total:0:01:33 | ETA:0:04:46 | Loss:0.00638824161625502 | top1:0.9925287356321839
175/689 Data:0.008 | Batch:0.560 | Total:0:01:34 | ETA:0:04:45 | Loss:0.006467624198661984 | top1:0.9924571428571428
176/689 Data:0.009 | Batch:0.556 | Total:0:01:34 | ETA:0:04:45 | Loss:0.006438191452037823 | top1:0.9925
177/689 Data:0.009 | Batch:0.551 | Total:0:01:35 | ETA:0:04:44 | Loss:0.0064022095727939475 | top1:0.9925423728813559
178/689 Data:0.010 | Batch:0.548 | Total:0:01:36 | ETA:0:04:44 | Loss:0.006368001602822915 | top1:0.9925842696629213
179/689 Data:0.009 | Batch:0.552 | Total:0:01:36 | ETA:0:04:43 | Loss:0.006383626491895372 | top1:0.9925698324022346
180/689 Data:0.014 | Batch:0.553 | Total:0:01:37 | ETA:0:04:42 | Loss:0.006

243/689 Data:0.009 | Batch:0.550 | Total:0:02:11 | ETA:0:04:06 | Loss:0.0063458259328287156 | top1:0.9926748971193415
244/689 Data:0.007 | Batch:0.554 | Total:0:02:12 | ETA:0:04:05 | Loss:0.006321752239437344 | top1:0.9927049180327869
245/689 Data:0.009 | Batch:0.550 | Total:0:02:12 | ETA:0:04:05 | Loss:0.006356131705207209 | top1:0.9926530612244898
246/689 Data:0.009 | Batch:0.551 | Total:0:02:13 | ETA:0:04:05 | Loss:0.006368644866538452 | top1:0.9926422764227643
247/689 Data:0.008 | Batch:0.566 | Total:0:02:13 | ETA:0:04:05 | Loss:0.006418608541633163 | top1:0.9925910931174089
248/689 Data:0.010 | Batch:0.555 | Total:0:02:14 | ETA:0:04:04 | Loss:0.006393496512036138 | top1:0.9926209677419355
249/689 Data:0.009 | Batch:0.553 | Total:0:02:15 | ETA:0:04:04 | Loss:0.006371054447219328 | top1:0.9926506024096385
250/689 Data:0.009 | Batch:0.552 | Total:0:02:15 | ETA:0:04:04 | Loss:0.006378156792809023 | top1:0.99264
251/689 Data:0.010 | Batch:0.554 | Total:0:02:16 | ETA:0:04:03 | Loss:0.00

314/689 Data:0.010 | Batch:0.576 | Total:0:02:53 | ETA:0:03:36 | Loss:0.006508169141626779 | top1:0.992484076433121
315/689 Data:0.013 | Batch:0.545 | Total:0:02:53 | ETA:0:03:36 | Loss:0.00648888992328134 | top1:0.9925079365079366
316/689 Data:0.025 | Batch:0.651 | Total:0:02:54 | ETA:0:03:35 | Loss:0.0064687223823080165 | top1:0.9925316455696203
317/689 Data:0.030 | Batch:0.633 | Total:0:02:54 | ETA:0:03:40 | Loss:0.006448873630243762 | top1:0.9925552050473186
318/689 Data:0.013 | Batch:0.594 | Total:0:02:55 | ETA:0:03:39 | Loss:0.006459040731376875 | top1:0.9925471698113207
319/689 Data:0.042 | Batch:0.627 | Total:0:02:56 | ETA:0:03:40 | Loss:0.006440176404731605 | top1:0.9925705329153605
320/689 Data:0.018 | Batch:0.580 | Total:0:02:56 | ETA:0:03:40 | Loss:0.00645225888101777 | top1:0.9925625
321/689 Data:0.011 | Batch:0.629 | Total:0:02:57 | ETA:0:03:39 | Loss:0.006432717106483857 | top1:0.9925856697819314
322/689 Data:0.021 | Batch:0.562 | Total:0:02:57 | ETA:0:03:39 | Loss:0.006

385/689 Data:0.011 | Batch:0.520 | Total:0:03:32 | ETA:0:02:27 | Loss:0.006269911226735967 | top1:0.9928571428571429
386/689 Data:0.025 | Batch:0.549 | Total:0:03:33 | ETA:0:02:27 | Loss:0.006255485620673482 | top1:0.9928756476683938
387/689 Data:0.010 | Batch:0.504 | Total:0:03:33 | ETA:0:02:31 | Loss:0.006242660255270553 | top1:0.9928940568475452
388/689 Data:0.019 | Batch:0.445 | Total:0:03:34 | ETA:0:02:31 | Loss:0.006230444135382509 | top1:0.9929123711340206
389/689 Data:0.016 | Batch:0.300 | Total:0:03:34 | ETA:0:02:30 | Loss:0.006214874680855435 | top1:0.9929305912596401
390/689 Data:0.011 | Batch:0.277 | Total:0:03:34 | ETA:0:02:14 | Loss:0.0062007135560400994 | top1:0.992948717948718
391/689 Data:0.010 | Batch:0.276 | Total:0:03:34 | ETA:0:02:14 | Loss:0.006186904644670934 | top1:0.9929667519181585
392/689 Data:0.015 | Batch:0.290 | Total:0:03:35 | ETA:0:02:13 | Loss:0.006185858489599775 | top1:0.9929336734693878
393/689 Data:0.009 | Batch:0.262 | Total:0:03:35 | ETA:0:02:13 |